# 1 Network Firewalls: Nftables


## Task 1: What is netfilter, and what are nftables?

Netfilter is a framework provided by the Linux kernel that allows various networking-related operations to be implemented in the form of customized handlers. It provides a set of hooks within the Linux kernel for intercepting and manipulating network packets.

nf_tables is a subsystem of the Linux kernel responsible for packet filtering, NAT and packet mangling. It replaces the existing systems such as iptables and provides a single, unified and efficient framework for handling IPv4, IPv6, ARP and other packet types.

## Task 2: What is the role of a table, chain, hook in nftables? What is the relationship between hook and priority in nftables?

In nftables, a table is a container for chains. A chain is a sequence of rules which are evaluated in order for each packet. A hook determines at which point in the packet processing pipeline a chain (and its rules) are evaluated.

The priority determines the order of evaluation when multiple hooks are registered at the same point in the packet processing pipeline. Lower priority values are processed first.

## Task 3: Forward a TCP and UDP port using nftables. external Address: 1234 <- > 127.0.0.1:4321.

```python
# Add the following rules to your nftables configuration
nft add rule ip filter forward tcp dport 1234 counter redirect to 4321
nft add rule ip filter forward udp dport 1234 counter redirect to 4321
```

## Task 4: Echo packets using nftables.

```python
# Add the following rules to your nftables configuration
nft add rule ip filter input tcp dport 9000 counter redirect to 1234
nft add rule ip filter output udp dport 9000 counter drop
```

## Task 5:
*Drop outgoing UDP packets via the default interface that contain the bytes 0xca 0xfe at position 100 (position in bytes, counted from the start of the UDP header). Which netfilter hook and priority do you need to use for that and why?*

```python
# Add the following rule to your nftables configuration
nft add rule ip filter output udp payload 100 2 0xcafe counter drop
```

## Task 6:
*Drop incoming TCP packets via the default interface that contains the bytes 0xbe 0xef at position 10000 (position in bytes, counted from the start of the TCP header). Which netfilter hook and priority do you need to use and why?*

```python
# Add the following rule to your nftables configuration
nft add rule ip filter input tcp payload 10000 2 0xbeef counter drop
```

# 2 Network Monitoring using Libpcap and Tcpdump

### 2.1.1 Simple Statistics

1. Count the number of packets in the network trace. For that, create the fundamental structure of your application to retrieve packets through libpcap and implement a counter to verify the general working.

In [4]:
import pcap

# task 2.1
def count_packets(pcap_file):
    packet_count = 0
    for timestamp, packet in pcap.pcap(pcap_file):
        packet_count += 1
    print(f"Total number of packets: {packet_count}")
    
pcap_file_path = 'sample1m.pcap'
count_packets(pcap_file_path)



Total number of packets: 1000000


2. Considering IPv4 only, count the number of packets, the number of IPs, and the amount of transmitted payload. Does the trace contain more network protocols? Identify all present network (layer 3) protocols and count the packets per protocol. Specifically for IPv4, parse the header to analyze the IP addresses and payload data.

In [5]:
import pcap
import struct

#task 2.2
def count_packets(pcap_file):
    ipv4_count = 0
    unique_ips = set()
    protocol_counts = {}
    total_payload = 0
    
    for timestamp, packet in pcap.pcap(pcap_file):
        #unpack ethernet header (6 bytes dest MAC, 6 bytes src MAC, 2 bytes protocol)
        eth_header = struct.unpack('!6s6sH', packet[:14]) 
        eth_protocol = eth_header[2]
        
        # check for IPv4 packets (0x0800)
        if eth_protocol == 0x0800:
            ipv4_count += 1
            ip_header = struct.unpack('!BBHHHBBH4s4s', packet[14:34])
            protocol = ip_header[6]
            src_ip = packet[26:30]
            dst_ip = packet[30:34]
            unique_ips.update([src_ip, dst_ip])
            
            # Calculate payload size
            total_length = ip_header[2]
            header_length = (ip_header[0] & 0xF) * 4
            payload_size = total_length - header_length
            total_payload += payload_size
            
            # Count packets per protocol
            if protocol in protocol_counts:
                protocol_counts[protocol] += 1
            else:
                protocol_counts[protocol] = 1
    
    print(f"IPv4 packets: {ipv4_count}")
    print(f"Unique IPs: {len(unique_ips)}")
    print(f"Total IPv4 payload: {total_payload} bytes")
    print("Packets per protocol:", protocol_counts)

pcap_file_path = 'sample1m.pcap'
count_packets(pcap_file_path)

IPv4 packets: 873244
Unique IPs: 235352
Total IPv4 payload: 699371757 bytes
Packets per protocol: {17: 92168, 6: 571537, 1: 196534, 47: 12570, 50: 10, 41: 425}


3. Considering IPv4 only, 
how many bytes of payloads are transmitted over TCP and UDP? 
Does the trace contain more transport protocols? 
Are there packets with inconsistency regarding their payload length? 
Identify all present transport (layer4) protocols over IPv4 and parse TCP/UDP packets to analyze the payload bytes.
Are there any indications that someone is deliberately sending unusual packets?

In [6]:
import pcap
import struct

tcp_payload_bytes = 0
udp_payload_bytes = 0
other_protocols = {}
inconsistencies = 0
unusual_packets = []

def parse_packet(timestamp, packet):
    global tcp_payload_bytes, udp_payload_bytes, inconsistencies
    
    # Ethernet header is 14 bytes long, IP header starts immediately after
    eth_header = struct.unpack("!6s6s2s", packet[:14])
    eth_type = eth_header[2]
    
    # Check if it's an IP packet (0x0800)
    if eth_type != b'\x08\x00':
        print("Non-IP Packet type not supported")
        return
    
    # IP header extraction
    ip_header = packet[14:34]  # IP header is after Ethernet header (14 bytes)
    iph = struct.unpack('!BBHHHBBH4s4s', ip_header)
    version_ihl = iph[0]
    ihl = version_ihl & 0xF
    iph_length = ihl * 4
    protocol = iph[6]
    total_length = iph[2]
    
    # Calculate payload size
    payload_size = total_length - iph_length
    
    # Check protocol and update counters
    if protocol == 6:  # TCP
        tcp_payload_bytes += payload_size
    elif protocol == 17:  # UDP
        udp_payload_bytes += payload_size
    else:
        if protocol not in other_protocols:
            other_protocols[protocol] = payload_size
        else:
            other_protocols[protocol] += payload_size
    
    # Example inconsistency check (simplified)
    if payload_size < 0:
        inconsistencies += 1
    
    # Example check for unusual packets (simplified)
    if payload_size > 10000:
        unusual_packets.append(packet)

# Open pcap file
pc = pcap.pcap(name='sample1m.pcap')
pc.setfilter('ip')  # Filter for IP packets only

# Parse each packet
for timestamp, packet in pc:
    parse_packet(timestamp, packet)

# Output findings
print(f"TCP Payload Bytes: {tcp_payload_bytes}")
print(f"UDP Payload Bytes: {udp_payload_bytes}")
print(f"Other Protocols: {other_protocols}")
print(f"Inconsistencies: {inconsistencies}")
if unusual_packets:
    print(f"Found {len(unusual_packets)} unusual packets.")

TCP Payload Bytes: 613657088
UDP Payload Bytes: 80256331
Other Protocols: {1: 2772733, 47: 2489746, 50: 1740, 41: 194119}
Inconsistencies: 0


4. Count the number of flows and identify the most chatty conversations on the network and transport layer.

In [14]:
import socket
import struct
import pcap

network_flows = {}
transport_flows = {}

def parse_packet(timestamp, packet):
    global network_flows, transport_flows
    
    # get ethernet header (6 bytes dest MAC, 6 bytes src MAC, 2 bytes protocol)
    eth_header = struct.unpack("!6s6s2s", packet[:14])
    eth_type = eth_header[2]
    
    if eth_type != b'\x08\x00':
        return  # Skip non-IP packets
    
    ip_header = packet[14:34]  # IP header is after Ethernet header (14 bytes)
    iph = struct.unpack('!BBHHHBBH4s4s', ip_header) # IP header format
    version_ihl = iph[0]
    ihl = version_ihl & 0xF
    iph_length = ihl * 4
    protocol = iph[6]
    total_length = iph[2]
    src_ip = socket.inet_ntoa(iph[8])
    dst_ip = socket.inet_ntoa(iph[9])
    
    # payload size = total length - header length
    payload_size = total_length - iph_length
    
    network_flow = (src_ip, dst_ip)
    # Update network flow with payload size
    network_flows[network_flow] = network_flows.get(network_flow, 0) + payload_size
    
    if len(packet) < 14 + iph_length + 4:
        return  # packet too short for TCP/UDP header
    
    # TCP or UDP
    if protocol == 6 or protocol == 17: 
        src_port, dst_port = struct.unpack('!HH', packet[14+iph_length:14+iph_length+4])
        transport_flow = (src_ip, src_port, dst_ip, dst_port, protocol)
        transport_flows[transport_flow] = transport_flows.get(transport_flow, 0) + payload_size


pc = pcap.pcap(name='sample1m.pcap')
pc.setfilter('ip')  # Filter for IP packets only

for timestamp, packet in pc:
    parse_packet(timestamp, packet)

# get maximum flows 
most_chatty_network_flow = max(network_flows, key=network_flows.get, default="No Flows")
most_chatty_transport_flow = max(transport_flows, key=transport_flows.get, default="No Flows")

# print results
print(f"Total Network Flows: {len(network_flows)}")
print(f"Most Chatty Network Flow: {most_chatty_network_flow} with {network_flows.get(most_chatty_network_flow, 0)} bytes")
print(f"Total Transport Flows: {len(transport_flows)}")
print(f"Most Chatty Transport Flow: {most_chatty_transport_flow} with {transport_flows.get(most_chatty_transport_flow, 0)} bytes")

Total Network Flows: 280626
Most Chatty Network Flow: ('23.26.144.53', '163.139.25.45') with 336173980 bytes
Total Transport Flows: 99992
Most Chatty Transport Flow: ('23.26.144.53', 80, '163.139.25.45', 18748, 6) with 195542719 bytes


5. Identify HTTP requests that submit login credentials via basic HTTP Auth. For
that, use the network trace illauth.pcap

In [16]:
import pcap
import socket
import struct

def parse_http_auth_credentials(pcap_file):
    pc = pcap.pcap(pcap_file)
    pc.setfilter('tcp port 80')  # Filter for HTTP traffic

    for timestamp, packet in pc:
        if len(packet) < 14 + 20 + 20:  # Ethernet + minimum IP header + minimum TCP header
            continue

        # Ethernet header is 14 bytes, IP header is 20 bytes minimum
        ip_header = packet[14:34]
        iph = struct.unpack('!BBHHHBBH4s4s', ip_header)
        iph_length = (iph[0] & 0xF) * 4

        # Needs to be long enough to contain the full IP header and a TCP header
        if len(packet) < 14 + iph_length + 20:
            continue

        tcp_header = packet[14+iph_length:14+iph_length+20]
        
        if len(tcp_header) < 20:
            continue

        tcph = struct.unpack('!HHLLBBHHH', tcp_header)
        tcp_length = (tcph[4] >> 4) * 4
        payload_start = 14 + iph_length + tcp_length
        payload = packet[payload_start:]

        try:
            payload = payload.decode('utf-8')
            headers = payload.split('\r\n')
            for header in headers:
                # check for Authorization header
                if header.startswith('Authorization: Basic '):
                    credentials = header.split(' ')[2]
                    print(f"Found Basic Auth Credentials (Base64): {credentials}")
                    break
        except UnicodeDecodeError:
            continue

parse_http_auth_credentials('illauth.pcap')

Found Basic Auth Credentials (Base64): aGJkYWlyeWU0OmNoZWVzZWNha2U=
Found Basic Auth Credentials (Base64): aGJkYWlyeWU0OmNoZWVzZWNha2U=


### 2.1.2 Attack Detection

Scenario 1: Identifying IP addresses that did not finish the three-way TCP handshake

In [20]:
import pcap
import socket
import struct

# Initialize data structures
tcp_handshakes = {}
incomplete_handshakes = {}

def update_handshake_state(packet, iph_length, src_ip, dst_ip):
    # Extract TCP header and flags
    tcp_header = packet[14+iph_length:14+iph_length+20]
    src_port, dst_port, _, _, tcp_flags = struct.unpack('!HHLLH', tcp_header[:14])
    syn_flag = (tcp_flags & 0x02) >> 1
    ack_flag = tcp_flags & 0x01
    
    # flow is defined by 4-tuple (src_ip, dst_ip, src_port, dst_port)
    flow_key = (src_ip, dst_ip, src_port, dst_port)
    
    # Update handshake state
    if syn_flag and not ack_flag:
        tcp_handshakes[flow_key] = 'SYN_SENT'
    elif syn_flag and ack_flag:
        tcp_handshakes[flow_key] = 'SYN_ACK_RECEIVED'
    elif not syn_flag and ack_flag and tcp_handshakes.get(flow_key) == 'SYN_SENT':
        tcp_handshakes[flow_key] = 'HANDSHAKE_COMPLETED'

def parse_packet(packet):
    # Parse IP header
    eth_header = struct.unpack("!6s6s2s", packet[:14])
    eth_type = eth_header[2]
    if eth_type != b'\x08\x00':  # Not an IP packet
        return
    
    ip_header = packet[14:34]
    iph = struct.unpack('!BBHHHBBH4s4s', ip_header)
    ihl = (iph[0] & 0xF) * 4
    protocol = iph[6]
    src_ip = socket.inet_ntoa(iph[8])
    dst_ip = socket.inet_ntoa(iph[9])

    
    if protocol == 6:  # TCP
        update_handshake_state(packet, ihl, src_ip, dst_ip)

pc = pcap.pcap('sample1m.pcap')

for timestamp, packet in pc:
    parse_packet(packet)

for flow, state in tcp_handshakes.items():
    if state == 'SYN_SENT':
        src_ip = flow[0]
        incomplete_handshakes[src_ip] = incomplete_handshakes.get(src_ip, 0) + 1

# get top 5 IP addresses
top_5_ips = sorted(incomplete_handshakes.items(), key=lambda x: x[1], reverse=True)[:5]

# Print results
print("Top 5 IP addresses with incomplete handshakes:")
for ip, count in top_5_ips:
    print(f"{ip}: {count} incomplete handshakes")

Top 5 IP addresses with incomplete handshakes:
125.129.161.32: 7229 incomplete handshakes
185.247.198.175: 2218 incomplete handshakes
5.247.225.198: 2024 incomplete handshakes
5.247.225.196: 2003 incomplete handshakes
5.247.225.197: 1965 incomplete handshakes


Scenario 2: Identifying IP addresses that receive an ICMP port un- reachable packet in response to initiating a UDP communication.

In [25]:
import pcap
import socket
import struct

icmp_unreachable_counts = {}
udp_flows = set() 

def parse_packet(packet):
    eth_header = struct.unpack("!6s6s2s", packet[:14])
    eth_type = eth_header[2]
    if eth_type != b'\x08\x00':  # Not an IP packet
        return
    
    ip_header = packet[14:34]
    iph = struct.unpack('!BBHHHBBH4s4s', ip_header)
    ihl = (iph[0] & 0xF) * 4
    protocol = iph[6]
    src_ip = socket.inet_ntoa(iph[8])
    dst_ip = socket.inet_ntoa(iph[9])
    
    if protocol == 17:  # UDP
        if len(packet) >= 14 + ihl + 8:
            udp_header = packet[14+ihl:14+ihl+8]
            src_port, dst_port = struct.unpack('!HH', udp_header[:4])
            # Add UDP flow to tracking
            udp_flows.add((src_ip, dst_ip, src_port, dst_port))
        else:
            pass # skip packets that are too short
        
    elif protocol == 1:  # ICMP
        icmp_header = packet[14+ihl:14+ihl+4]
        icmp_type, code = struct.unpack('!BB', icmp_header[:2])
        if icmp_type == 3 and code == 3:  # ICMP port unreachable
            original_ip_header = packet[14+ihl+8:14+ihl+8+20]
            original_iph = struct.unpack('!BBHHHBBH4s4s', original_ip_header)
            original_src_ip = socket.inet_ntoa(original_iph[8])
            original_dst_ip = socket.inet_ntoa(original_iph[9])
            original_udp_header = packet[14+ihl+8+20:14+ihl+8+20+8]
            original_src_port, original_dst_port = struct.unpack('!HH', original_udp_header[:4])
            
            # Check if this ICMP packet corresponds to a tracked UDP flow
            if (original_dst_ip, original_src_ip, original_dst_port, original_src_port) in udp_flows:
                # Increment count 
                icmp_unreachable_counts[dst_ip] = icmp_unreachable_counts.get(dst_ip, 0) + 1

pc = pcap.pcap('sample1m.pcap')

for timestamp, packet in pc:
    parse_packet(packet)

sorted_ips = sorted(icmp_unreachable_counts.items(), key=lambda x: x[1], reverse=True)
top_5_ips = sorted_ips[:5]

# Print results
print("Top 5 IP addresses with ICMP port unreachable packets in response to UDP:")
for ip, count in top_5_ips:
    print(f"{ip}: {count}")

Top 5 IP addresses with ICMP port unreachable packets in response to UDP:
163.139.103.125: 5
79.150.193.228: 2
13.144.197.61: 2


# Tcpdump 

1. To read the given pcap file and write the first 10000 packets to another capture file we used

```
# tcpdump -r sample1m.pcap -w sample.pcap -c 10000

```
2. Then, to verify the number of packets in the new capture, we executed

```
# tcpdump -r sample.pcap -n | wc -l
```

3. To list the top five talkers in the new sample we called

```
# tcpdump -nnr sample.pcap | awk -F" " '{print $3}' | sort | uniq -c | sort -nr | head -5
```
Which yields 
```
2808 23.26.144.53.80
1324 203.194.219.227
 463 183.175.123.12.443
 434 104.134.68.10.443
 194 2001:fdc7:6c7:e2d0:e009:bf8e:ff79:a6d2.50695
```

4. To list packets with TCP Reset Flags we used

```
# tcpdump -nnr sample.pcap 'tcp[13] & 4 != 0'
```
Which outputs 
```
06:00:00.393431 IP 133.7.179.211.1688 > 125.129.161.32.37910: Flags [R.], seq 0, ack 2361643466, win 0, length 0
06:00:00.398695 IP 5.247.201.219.40447 > 203.194.205.99.8088: Flags [R], seq 3742792195, win 1200, length 0
06:00:00.399092 IP 133.7.227.119.1505 > 5.50.89.204.50555: Flags [R.], seq 0, ack 3035865952, win 0, length 0
06:00:00.404464 IP 176.183.7.18.48119 > 133.238.15.5.59723: Flags [R], seq 503064290, win 1200, length 0
06:00:00.405412 IP 203.194.195.84.11211 > 196.59.215.189.59175: Flags [R.], seq 0, ack 3429023779, win 0, length 0
06:00:00.417526 IP 133.7.209.189.1688 > 125.129.161.32.33669: Flags [R.], seq 0, ack 1161876995, win 0, length 0
06:00:00.421212 IP 133.238.12.81.8545 > 206.189.117.123.35886: Flags [R.], seq 0, ack 1086435738, win 65535, length 0
06:00:00.432755 IP 203.194.193.23.32089 > 91.154.198.49.80: Flags [R.], seq 1168525437, ack 3682231626, win 60, length 0
06:00:00.432780 IP 203.194.193.23.32089 > 91.154.198.49.80: Flags [R.], seq 0, ack 911, win 60, length 0
06:00:00.437393 IP 5.247.225.196.54273 > 202.74.62.220.22376: Flags [R], seq 3740515204, win 1200, length 0
06:00:00.437478 IP 203.194.194.208.33880 > 194.19.145.252.52457: Flags [R.], seq 0, ack 33509389, win 0, length 0
06:00:00.441181 IP 176.183.5.228.40408 > 203.194.193.254.58097: Flags [R], seq 559880004, win 1200, length 0
06:00:00.442948 IP 133.238.14.63.1688 > 125.129.161.32.50644: Flags [R.], seq 0, ack 1755134450, win 65535, length 0
06:00:00.443688 IP 133.7.138.32.464 > 5.50.89.205.50469: Flags [R.], seq 0, ack 2257123250, win 0, length 0
06:00:00.443936 IP 49.150.51.188.59216 > 163.139.51.41.993: Flags [R], seq 271487112, win 0, length 0
06:00:00.446376 IP 5.247.225.198.53972 > 203.194.206.213.19315: Flags [R], seq 1026780058, win 1200, length 0
06:00:00.450353 IP 133.238.14.154.23 > 209.244.217.224.29187: Flags [R.], seq 0, ack 2240416411, win 18512, length 0
06:00:00.459866 IP 5.247.201.219.40447 > 203.194.205.10.8088: Flags [R], seq 2019600399, win 1200, length 0
06:00:00.467766 IP 5.247.201.219.40447 > 203.194.206.19.8088: Flags [R], seq 429684133, win 1200, length 0
```

5. To only capture HTTP data packets on port 80 and avoid capturing the TCP session setup packets
```
# tcpdump -nnr sample.pcap 'tcp port 80 and (tcp[13] & 7 == 0)'
```

Output:
```
reading from PCAP-NG file sample.pcap
06:00:00.392892 IP 17.139.176.81.80 > 163.139.101.107.51153: Flags [.], seq 296862263:296863637, ack 2208581056, win 118, options [nop,nop,TS val 925115841 ecr 798460735], length 1374: HTTP
06:00:00.392909 IP 17.139.176.81.80 > 163.139.101.107.51153: Flags [.], seq 1374:2748, ack 1, win 118, options [nop,nop,TS val 925115841 ecr 798460735], length 1374: HTTP
06:00:00.393110 IP 17.139.176.81.80 > 163.139.101.107.51153: Flags [.W], seq 16488:17862, ack 1, win 118, options [nop,nop,TS val 925115841 ecr 798460736], length 1374: HTTP
[...]
06:00:00.468770 IP 23.26.144.53.80 > 163.139.25.45.18727: Flags [.], seq 4094689:4096149, ack 1, win 229, length 1460: HTTP
06:00:00.468787 IP 23.26.144.53.80 > 163.139.25.45.18727: Flags [.], seq 4096149:4097609, ack 1, win 229, length 1460: HTTP
```

6. A simple approach to detect the presence of port scans using tcpdump, can be done by counting the number of SYN's and compare them to the number of SYN-ACK's. 
Count SYN:
```
# tcpdump -nnr sample.pcap 'tcp[13] & 2 != 0' | wc -l
```
Output: 
```
666
```

Count SYN-ACK:
```
tcpdump -nnr sample.pcap 'tcp[13] = 18' | wc -l
```
Output:
```
61
```
This indicates a SYN scan is present in the sample.